In [1]:
import glob
import pandas as pd
import numpy as np



pd.set_option('display.max_columns', None)




In [2]:
                               
demand = pd.read_csv(r'C:\Users\hudaya416\OneDrive - Comcast\Project QA docs\SCE\Demand\SPOKE_demand.csv')
fcast = pd.read_csv(r'C:\Users\hudaya416\OneDrive - Comcast\Project QA docs\SCE\Forecast\Spoke_forecast.csv')

#demand['demand'] = demand.demand.str.replace(',','')
#demand['demand'].astype(np.float).astype("Int32")
#demand['demand'] = demand.demand.astype(np.int64)


C:\Users\hudaya416\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df1 = demand.merge(fcast[['Week_Num', 'site_id', 'site_region', 'forecast_group', 'site_forecast']], how = 'left',
                left_on = ['Week_Num', 'site_id', 'forecast_group'], 
                right_on = ['Week_Num', 'site_id', 'forecast_group'])
df1.head()


Week_Num           site_id site_region_x             forecast_group demand  \
0  202001.0  BSRINTRANSITLOST     Big South      INDOOR-OUTDOOR CAMERA     13   
1  202001.0  BSRINTRANSITLOST     Big South                        XB3      2   
2  202101.0    Cable East Inc     Big South            WIRELESS ROUTER    198   
3  202001.0           CCBS003     Big South    BUSINESS NETWORK CAMERA     84   
4  202001.0           CCBS003     Big South  BUSINESS WIRELESS GATEWAY    210   

  site_region_y site_forecast  
0           NaN           NaN  
1           NaN           NaN  
2     Big South            16  
3           NaN           NaN  
4           NaN           NaN

In [4]:
df1 = df1[['Week_Num', 'site_id', 'site_region_x', 'forecast_group', 'site_forecast' , 'demand']]
df1 = df1.rename(columns={"site_region_x": "site_region"})
df2 = fcast.merge(demand[['Week_Num', 'site_id', 'forecast_group', 'demand']], how = 'left',
                left_on = ['Week_Num', 'site_id', 'forecast_group'], 
                right_on = ['Week_Num', 'site_id', 'forecast_group'])
df2.head()

Week_Num site_class site_region  site_id           forecast_group  \
0    202040      Spoke     Beltway   SEBE39     APPLIANCE CONTROLLER   
1    202040      Spoke     Beltway   SEBE07         APPLIANCE MODULE   
2    202040      Spoke     Beltway   SEBE39         APPLIANCE MODULE   
3    202040      Spoke     Beltway  CNBE107  BUSINESS NETWORK CAMERA   
4    202040      Spoke     Beltway  CNBE108  BUSINESS NETWORK CAMERA   

  site_forecast demand  
0             1    NaN  
1             1      2  
2             8    NaN  
3             4    NaN  
4             1    NaN

In [5]:
df2 = df2[['Week_Num', 'site_id', 'site_region', 'forecast_group', 'site_forecast', 'demand']]
df3 = pd.concat([df1,df2], ignore_index=False)
df3['concat'] = df3['Week_Num'].astype(str) + "+" + df3['site_id'] + "+" + df3['forecast_group']
df3 = df3.drop_duplicates(subset='concat', keep="first").drop(columns=['concat'])
df3 = df3.fillna(0)
demand['demand'].astype(np.float).astype("Int32")

In [6]:
df3.head()

Week_Num           site_id site_region             forecast_group  \
0  202001.0  BSRINTRANSITLOST   Big South      INDOOR-OUTDOOR CAMERA   
1  202001.0  BSRINTRANSITLOST   Big South                        XB3   
2  202101.0    Cable East Inc   Big South            WIRELESS ROUTER   
3  202001.0           CCBS003   Big South    BUSINESS NETWORK CAMERA   
4  202001.0           CCBS003   Big South  BUSINESS WIRELESS GATEWAY   

  site_forecast demand  
0             0     13  
1             0      2  
2            16    198  
3             0     84  
4             0    210

In [7]:
df3['diff'] = df3['site_forecast'] - df3['demand']
df3['tAPE_1'] = np.where(df3['site_forecast'] == df3['demand'], 0.0, df3['diff'].abs())
df3['tAPE_2'] = np.where(df3['demand'] == 0.0, 1.0, df3['diff'].abs()/df3['demand']) 
df3['tAPE'] = np.around(df3[['tAPE_1','tAPE_2']].min(axis=1), 2)
df4 = df3.drop(columns = ['tAPE_1', 'tAPE_2'])
df4.head()
df4 = df4.sort_values(['Week_Num'],ascending=True).reset_index().drop(columns='index')
df4['site_class'] = 'SPOKE'
df4.head()




TypeError: unsupported operand type(s) for -: 'int' and 'str'

In [ ]:
df4.to_csv(r'C:\Users\hudaya416\OneDrive - Comcast\Project QA docs\SCE\Accuracy_Metrics_spokesup.csv', index = False)